<img src='img/dockerlogo.png' width=800></img>
# <center><a>Создание docker контейнеров</a>

___
## $\boxed{\text{Nginx}}$

### Файл конфигурации
- `events`: параметры количества подключений и worker-потоков
- `http`: параметры для обработки HTTP-запросов
    - `server`: параметры сервера
        - `listen`: номер порта, на котором сервер принимает соединения
        - `server_name`: имя сервера, которое он будет обслуживать
        - `location`: параметры обработки запросов для указанного пути
            - `limit_except POST`: инструкции внутри этого блока применяются ко всем HTTP-методам, кроме POST
                - `deny all`: запрещает доступ для всех запросов, кроме POST, что приводит к отклонению всех запросов, кроме POST, с кодом ответа 403 Forbidden
            - `root`: корневая директория для статических файлов, которые будут отправляться в ответ
            - `index`: указывает, какой файл по умолчанию отправляется в ответ на запросы в корневую директорию, если запрос не указывает конкретный файл

In [ ]:
events {

}

http {
server {
listen 80;
server_name localhost;

        location / {
            limit_except POST {
                deny all;
            }
            root /usr/share/nginx/html;
            index index.html;
        }
    }

}

### Dockerfile

- Базовый образ
- Копирование локального файла конфига в директорию контейнера
- Сообщает, что контейнер должен прослушивать запросы на порту 80
- Запуск nginx
    - -g daemon off : говорит Nginx работать без фонового режима. Это важно в контейнерах, так как, если основной процесс завершится, контейнер остановится

In [ ]:
FROM nginx:alpine
COPY nginx.conf /etc/nginx/nginx.conf
EXPOSE 80
CMD ["nginx", "-g", "daemon off;"]

### Сборка и запуск
- -t: создание имени образа
- -f: использование кастомного названия Dockerfile

In [ ]:
sudo docker build -t nginx_hw -f Dockerfile_nginx .

- -d : запуск в фоновом режиме
- -p : сопоставление порта на хосте с портом в контейнере

In [ ]:
sudo docker run -d -p 80:80 nginx_hw

___
## $\boxed{\text{Postrgres}}$

### Dockerfile

- Базовый образ последней версии по умолчанию
- Создает переменный окружения, которые отвечают за создание пользователя, пароля и базы данных при запуске контейнера
- Сообщает, что контейнер должен прослушивать запросы на порту 5432

In [ ]:
FROM postgres

ENV POSTGRES_USER=test
ENV POSTGRES_PASSWORD=test
ENV POSTGRES_DB=test

EXPOSE 5432

### Сборка и запуск

In [ ]:
sudo docker build -t psql_hw -f Dockerfile_psql .

- --name: назначение имени контейнера

In [ ]:
sudo docker run -d -p 5432:5432 --name psql_cont psql_hw

- Подключение к БД по заданному в dockerfile логину и паролю 

In [ ]:
sudo docker exec -it psql_cont  psql -U test -d test